In [104]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import json
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [105]:
races = pd.read_csv(path+'data/races.csv')

rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

In [106]:
# query results API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/results.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races'][0]['Results']:
        for n in item:
            pass
            # print(f'{n} : {item[n]}')
        break
    break

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [107]:
laps = {
    'season': [],
    'round': [],
    'lap': [],
    'driver': [],
    'position': [],
    'time': []
}

In [108]:
# query results API

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
        url = 'http://ergast.com/api/f1/{}/{}/laps.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races']:
            for lap in item['Laps']:
                for driver in lap['Timings']:
                    laps['season'].append(item['season'])
                    laps['round'].append(item['round'])
                    laps['lap'].append(lap['number'])
                    laps['driver'].append(driver['driverId'])
                    laps['position'].append(driver['position'])
                    laps['time'].append(driver['time'])


laps = pd.DataFrame(laps)
laps.head()

,season,round,lap,driver,position,time
0,2014,1,1,rosberg,1,1:42.038
1,2014,1,1,ricciardo,2,1:43.549
2,2014,1,1,kevin_magnussen,3,1:45.402
3,2014,1,1,hamilton,4,1:46.128
4,2014,1,1,hulkenberg,5,1:46.986


In [109]:
laps.tail()

,season,round,lap,driver,position,time
5415,2022,21,2,vettel,8,1:51.802
5416,2022,21,2,gasly,9,1:52.007
5417,2022,21,2,mick_schumacher,10,1:51.772
5418,2022,21,2,bottas,11,1:51.493
5419,2022,21,2,stroll,12,1:51.889


In [110]:
qual = {
    'season': [],
    'round': [],
    'position': [],
    'circuit_id':[],
    'driver': [],
    'q1': [],
    'q2': [],
    'q3': []
}

In [111]:
# query results API

for i in rounds:
    url = 'http://ergast.com/api/f1/{}/{}/qualifying.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        for n in item['QualifyingResults']:
            qual['season'].append(2021)
            qual['round'].append(i)
            qual['position'].append(n['position'])
            qual['driver'].append(n['Driver']['driverId'])
            qual['circuit_id'].append(item['Circuit']['circuitId'])

            try:
                qual['q3'].append(n['Q3'])
            except:
                qual['q3'].append('OUT')

            try:
                qual['q2'].append(n['Q2'])
            except:
                qual['q2'].append('OUT')

            try:
                qual['q1'].append(n['Q1'])
            except:
                qual['q1'].append('DNF')

pd.DataFrame(qual)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# query API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/constructorStandings.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
        for n in item:
            print(f'{n} : {item[n]}')
        break

In [ ]:
# query API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/driverStandings.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
        for n in item:
            print(f'{n} : {item[n]}')
        break

In [ ]:
# query API

schedules = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'date': []
        }

for i in rounds:

    url = 'http://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(2021))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        schedules['season'].append(int(item['season']))
        schedules['round'].append(int(item['round']))
        schedules['circuit_id'].append(item['Circuit']['circuitId'])
        schedules['date'].append(datetime.strptime(item['date']+ 'T' +item['time'], '%Y-%m-%dT%H:%M:%SZ'))

schedules = pd.DataFrame(schedules)
schedules.head()